In [2]:
import sqlite3
import pandas as pd

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
 
    return conn

def getNews(conn):
    """
    :param conn: the Connection object
    :tablename
    :return: df
    """
    query = '''
    select 
        a.id,
	    a.symbol, 
	    a.descript, 
        a.exchange
    from 
	    stock_master as a
    '''
    return pd.read_sql_query(query, conn)

conn = create_connection("./data/test.db")
df = getNews(conn)
df.head()

,ID,symbol,descript,exchange
0,21480311,A,ARMOR MINERALS INC.,TOR-V
1,96768,AC,AIR CANADA,TOR
2,26050555,AD,ALARIS ROYALTY CORP,TOR
3,31349,AF,ALARMFORCE,TOR
4,69675825,AI,ATRIUM MORTGAGE INVESTMENT CORPORATION,TOR


In [3]:
set(df.exchange)

{'TOR', 'TOR-V'}

In [4]:
def get_ticket(symbol, exchange):
    ticket = ''
    if not ('.' in symbol):
        if exchange == "TOR-V":
            ticket = symbol + ".V"
        else:
            ticket = symbol + ".TO"
    return ticket

ticket = [get_ticket(s,x) for (s,x) in zip(df.symbol, df.exchange)]
df['ticket'] = ticket
df.head(10)

,ID,symbol,descript,exchange,ticket
0,21480311,A,ARMOR MINERALS INC.,TOR-V,A.V
1,96768,AC,AIR CANADA,TOR,AC.TO
2,26050555,AD,ALARIS ROYALTY CORP,TOR,AD.TO
3,31349,AF,ALARMFORCE,TOR,AF.TO
4,69675825,AI,ATRIUM MORTGAGE INVESTMENT CORPORATION,TOR,AI.TO
5,32660811,AR,ARGONAUT GOLD LTD,TOR,AR.TO
6,24512696,AZ,ARIZONA MINING INC,TOR,AZ.TO
7,21503032,AAB,ABERDEEN INTL INC,TOR,AAB.TO
8,29945393,AAV,ADVANTAGE OIL & GAS LTD,TOR,AAV.TO
9,13481030,ABT,ABSOLUTE SOFTWARE,TOR,ABT.TO


In [12]:
import requests
json_txts = []
sectors = []
industrys = []
for ticket in df.ticket:
    json_txt = ''
    sector = ''
    industry = ''
    if len(ticket) > 0:
        try:
            r = requests.get("https://query1.finance.yahoo.com/v10/finance/quoteSummary/" + ticket + "?lang=en-US&region=US&modules=assetProfile&corsDomain=finance.yahoo.com")
            json_txt = r.content
            try:
                sector = r.json()['quoteSummary']['result'][0]['assetProfile']['sector']
            except:
                pass
            try:
                industry = r.json()['quoteSummary']['result'][0]['assetProfile']['industry']
            except:
                pass
        except:
            pass    
    print(ticket + ':' + industry + ":" + sector)
    json_txts.append(json_txt)
    sectors.append(sector)
    industrys.append(industry)


& Mining:Basic Materials
NGH.V::
::
TTR.V:Integrated Freight & Logistics:Industrials
CPA.V::
::
::
TVR.V::
CXV.V::
XI.V::
::
IMT.V:Other Industrial Metals & Mining:Basic Materials
MON.V:Other Industrial Metals & Mining:Basic Materials
SDZ.V::
::
EHT.V:Solar:Technology
::
::
GBD.V::
BUS.V:Auto Manufacturers:Consumer Cyclical
::
NOU.V:Other Industrial Metals & Mining:Basic Materials
SGB.V::
::
QXP.V::
GAR.V::
KWA.V::
SME.V:Other Industrial Metals & Mining:Basic Materials
::
::
EEN.V::
::
TWM.V::
::
BEE.V:Agricultural Inputs:Basic Materials
::
::
::
::
::
::
::
::
::
SYN.V::
ISSUV.V::
ISSUH.V::
B.V:Other Industrial Metals & Mining:Basic Materials
BR.TO:Beverages—Brewers:Consumer Defensive
BX.TO::
BCB.TO:Beverages—Non-Alcoholic:Consumer Defensive
BCE.TO:Telecom Services:Communication Services
BDT.TO:Engineering & Construction:Industrials
BIR.TO:Oil & Gas E&P:Energy
BKX.TO:Oil & Gas E&P:Energy
BRB.TO::
BRY.TO:Oil & Gas Equipment & Services:Energy
BSC.TO::
BSX.TO:Gold:Basic Materials
BTO.TO:

In [16]:
df['industry']=industrys
df['sector'] = sectors
df['json'] = json_txt

In [18]:
print(len(set(industrys)))
print(len(set(sectors)))
df.head()

125
13


,ID,symbol,descript,exchange,ticket,industry,sector,json
0,21480311,A,ARMOR MINERALS INC.,TOR-V,A.V,Other Industrial Metals & Mining,Basic Materials,"b'{""quoteSummary"":{""result"":[{""assetProfile"":{..."
1,96768,AC,AIR CANADA,TOR,AC.TO,Airlines,Industrials,"b'{""quoteSummary"":{""result"":[{""assetProfile"":{..."
2,26050555,AD,ALARIS ROYALTY CORP,TOR,AD.TO,Conglomerates,Industrials,"b'{""quoteSummary"":{""result"":[{""assetProfile"":{..."
3,31349,AF,ALARMFORCE,TOR,AF.TO,,,"b'{""quoteSummary"":{""result"":[{""assetProfile"":{..."
4,69675825,AI,ATRIUM MORTGAGE INVESTMENT CORPORATION,TOR,AI.TO,Mortgage Finance,Financial Services,"b'{""quoteSummary"":{""result"":[{""assetProfile"":{..."


In [19]:
df.to_sql('master_ext', con = conn)